Привет! меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:


<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ..
</div>
<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Спасибо за  качественную работу, мне она понравилась!
- Работа очень лаконичная, было приятно ее читать.
- Я оставил некоторые советы надеюсь они будут полезными или интересными.
- Есть маленький недочет, но ты его поправишь за минуту =)


<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление[2]</font>
* Спасибо за оперативность
- Приятно видеть фидбек на комментарии.
- Недочеты исправлены - работа полностью корректна
- Рад был быть полезен. Удачи в дальнейшем обучении!

# Хомкин Константин, когорта 53DS
Проект "Машинное обучение в бизнесе", v.2.0 15.12.2022

# Введение
Мы работаем в добывающей компании «ГлавРосГосНефть». Необходимо на основе анализа имеющейся  информации принять решение о том, где бурить новую скважину.

Для выбора новой локации необходимо:
Собрать по выбранномым регионам основные характеристики скважин: качество нефти и объём её запасов;
Построить модель предсказания объёма запасов в новых скважинах;
Выбрать скважины с самыми высокими оценками значений;
Определить регион с максимальной суммарной прибылью отобранных скважин.

Для анализа нам доступны данные о пробах нефти в трёх регионах и характеристики для каждой скважины. 

В проекте построим модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой Bootstrap.

# Шаг 1. Импортируем необходимые библиотеки, откроем и изучим структуру доступных файлов

In [1]:
import pandas as pd # библиотека Pandas нам нужна для работы с датасетом

from sklearn.model_selection import train_test_split # функция train_test_split нужна для разделения данных на выборки

from sklearn.preprocessing import StandardScaler # метод стандартизации масштаба данных

from sklearn.linear_model import LinearRegression # модель "Линейная регрессия"

from sklearn.metrics import mean_squared_error # метод для расчета среднеквадратичной ошибки

import numpy as np # библиотека Numpy нужна для быстрого поиска максимальных значений



<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  на месте
</div>



**Поскольку у нас три идентичных датасета, необходимо трижды повторять все манипуляции с каждым датасетом. При этом количество датасетов "на входе" в проект может измениться. Поэтому, для 1) оптимизации длины кода и 2) универсализации кода для разного количества данных на входе объединим все данные в один объект - словарь. Изменяя лишь переменную с номерами файлов мы сможет масштабировать проект до любого адекватного количества данных о скважинах.**

**При этом, с точки зрения абсолютой длины кода, также всё нормально: даже если какая-то операция с датасетом выполняется "в одну строку", например извелечение признаков, значит для трех датасетов нужно повторить эту строку трижды, тогда как с использованием словаря и цикла мы "умещаемся" всего в две строки, что и будет видно ниже в коде.**

<div class="alert alert-block alert-success">
<b>Успех:</b> Отличное решение
</div>


In [2]:
# Объявляем переменную, где будем хранить уникальные признаки разных датасетов, в данном случае - номера файлов
regions = [0, 1, 2]  

# датасеты "сложим" в один словарь и прочитаем файлы одной строкой с внутренним циклом
df = {reg:pd.read_csv('geo_data_'+str(reg)+'.csv') for reg in regions}


In [3]:
# выводим первые 5 строк каждого датасета
for i in range(len(regions)):
    print(df[i].head(5)) 


      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


данные выглядят структурированными, потребуется удаление неинформативного столбца с id скважины, а также масштабирование признаков.

In [4]:
# Проверим данные на дубликаты
for i in range(len(regions)):
    print(df[i][df[i].duplicated()])


Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []
Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []
Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []


Полных дубликатов нет. Однако, стоит проверить отдельно по столбцу id, возможно есть записи под одинаковыми именами

In [5]:
# Выведем на экран разницу общего кол-ва строк и количества уникальных id
for i in range(len(regions)):
    print('Дубликатов по id в регионе', i, '  :', pd.Series(df[i]['id']).count()-pd.Series(df[i]['id'].unique()).count())

Дубликатов по id в регионе 0   : 10
Дубликатов по id в регионе 1   : 4
Дубликатов по id в регионе 2   : 4


Бинго! Есть под одинаковыми id по несколько записей. При этом это не полные дубликаты, то есть данные в остальных (кроме id) столбцах отличаются. Какие именно строки истинные - уже не выяснить, поэтому для снижения ошибок моделей данные под совпадающими id из датафрейма удалим.

In [6]:
# Удалим все дубликаты по id, не сохраняя никакую запись (аргумент keep=False)
for i in range(len(regions)): 
    df[i] = df[i].drop_duplicates(subset='id', keep=False, inplace=False)

In [7]:
# Еще раз выведем на экран разницу общего кол-ва строк и количества уникальных id

for i in range(len(regions)):
    print('Дубликатов по id в регионе', i, '  :', pd.Series(df[i]['id']).count()-pd.Series(df[i]['id'].unique()).count())

Дубликатов по id в регионе 0   : 0
Дубликатов по id в регионе 1   : 0
Дубликатов по id в регионе 2   : 0


Дубликатов не осталось. Теперь несущественый для модели признак можно удалить

In [8]:
# удаляем несущественный признак
for i in range(len(regions)):
    df[i]= df[i].drop(['id'], axis=1)


Несущественные данные удалены. Проверим таблицы на полноту

In [9]:
for i in range(len(regions)):
    print(df[i].info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 99980 entries, 0 to 99999
Data columns (total 4 columns):
f0         99980 non-null float64
f1         99980 non-null float64
f2         99980 non-null float64
product    99980 non-null float64
dtypes: float64(4)
memory usage: 3.8 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99992 entries, 0 to 99999
Data columns (total 4 columns):
f0         99992 non-null float64
f1         99992 non-null float64
f2         99992 non-null float64
product    99992 non-null float64
dtypes: float64(4)
memory usage: 3.8 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99992 entries, 0 to 99999
Data columns (total 4 columns):
f0         99992 non-null float64
f1         99992 non-null float64
f2         99992 non-null float64
product    99992 non-null float64
dtypes: float64(4)
memory usage: 3.8 MB
None


In [10]:
# Поскольку планируем использовать линейные модели, посмотрим на корреляции признаков:

for i in range(len(regions)):
    print(df[i].corr())

               f0        f1        f2   product
f0       1.000000 -0.440724 -0.003204  0.143504
f1      -0.440724  1.000000  0.001783 -0.192338
f2      -0.003204  0.001783  1.000000  0.483628
product  0.143504 -0.192338  0.483628  1.000000
               f0        f1        f2   product
f0       1.000000  0.182263 -0.001821 -0.030534
f1       0.182263  1.000000 -0.002608 -0.010167
f2      -0.001821 -0.002608  1.000000  0.999397
product -0.030534 -0.010167  0.999397  1.000000
               f0        f1        f2   product
f0       1.000000  0.000501 -0.000454 -0.001974
f1       0.000501  1.000000  0.000763 -0.001046
f2      -0.000454  0.000763  1.000000  0.445873
product -0.001974 -0.001046  0.445873  1.000000



<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения 

Изучеы корреляции признаков. По региону "2" всё отлично, корреляции минимальные. В регионе "0" есть заметная отрицательная корреляция признаков f0 и f1 (-0.44), а также признака f2 и целевого признака product (0.48). 
А вот с датасетов региона "1" ситуация сложнее. Высокая корреляция признака f2 и целевого признака product, значение практически единица. Это говорит о том, что имеется сильно выраженная прямопропорциональная зависимость между признаком f2 и целевым признаком. По условиям проекта - все признаки значимы и удалять мы их не будем. 
</div>


<div class="alert alert-block alert-success">
<b>Успех[2]:</b> Есть контакт!

**Промежуточный вывод:** Данные загружены, проверены на полноту и сотоветствие типов, несущественные данные (id) удалены. Требуется масштабирование, которое будет проведено после раздедения данных на выборки, во избежание эффекта утечки данных.

# Шаг 2. Обучение и проверка модели 

Разбьем данные на обучающую и валидационную выборки в соотношении 75:25.<br>
Обучим модель и сделаем предсказания на валидационной выборке.<br>
Сохраним предсказания и правильные ответы на валидационной выборке.<br>
Рассчитаем средний запас предсказанного сырья и RMSE модели.




<div class="alert alert-block alert-success">
<b>Успех:</b> Данные осмотрены, но можно было бы еще как минимум изучить корреляции, так как мы планируем использовать линейные модели, а для них это важно. 
</div>


<div class="alert alert-block alert-info">
<b>Совет </b>  Советую обратить внимание на библиотеки sweetviz и pandas_profiling помогут в проведении более тщательного EDA анализа. Исследовательский анализ можно делать и с помощью ручного вызова функций дефолтных библиотек. Данные библиотеки хороши для максимизации комфорта презентации результатов анализа бизнес-пользователям.  
Очень просты в использоовании, на вход кладется датафрейм: pandas_profiling.ProfileReport(df) 
</div>

In [11]:
# Обявляем переменные (также словари) для хранения выборок
features = {}
target = {}

# Делим датасеты на выборки и "складываем" в одноименные переменные 
for i in range(len(regions)):
    features[i]= df[i].drop(['product'], axis=1) # извлекаем признаки датасета i
    target[i] = df[i]['product'] # извлекаем целевой признак датасета i

In [12]:
# Обявляем переменные (также словари) для хранения обучающих и валидационных выборок
features_train = {}
features_valid = {}
target_train = {}
target_valid = {}


# отделяем 25% данных для "валидационной" выборки 
for i in range(len(regions)):
    features_train[i], features_valid[i], target_train[i], target_valid[i] = train_test_split(
        features[i], target[i], test_size=0.25, random_state=12345) 

In [13]:
# Проведем масштабирование признаков

scaler = StandardScaler() # объявляем метод

for i in range(len(regions)):
    scaler.fit(features_train[i]) # Обучаем метод только на обучающей выборке
    features_train[i] = pd.DataFrame(scaler.transform(features_train[i]), columns=features_train[i].columns) # Преобразуем данные и сохраняем в новой переменной в структуре DataFrame()
    features_valid[i] = pd.DataFrame(scaler.transform(features_valid[i]), columns=features_valid[i].columns) # Преобразуем данные и сохраняем в новой переменной в структуре DataFrame()


In [14]:
# Обучим модель "Линейная регрессия" для каждого датасета отдельно

# Объявляем необхожимые переменные
model = {}
predictions_valid = {}
result = {}
zapas_true = {}
zapas_pred = {}

# Инициализирем модель LinearRegression три раза так как обучать будем каждую на отдельной выборке
for i in range(len(regions)):
    model[i] = LinearRegression()

# Обучаем каждую модель отдельно на своей выборке, рассчитываем предсказания и RMSE
for i in range(len(regions)):
    model[i].fit(features_train[i], target_train[i]) # обуаем модель на тренировочной выборке
    predictions_valid[i] = model[i].predict(features_valid[i]) # получаем предсказания модели на валидационной выборке
    result[i] = mean_squared_error(target_valid[i], predictions_valid[i])**0.5 # получаем значение метрики RMSE на валидационной выборке
    zapas_pred[i] = predictions_valid[i].mean()  # расчет среднего запаса сырья по предсказанию
    zapas_true[i] = target_valid[i].mean() # расчет реального среднего запаса сырья
    print(f'Регион {i}: Средний запас сырья на скважину {zapas_pred[i]:.5f}/предс/, {zapas_true[i]:.5f}/реал/, RMSE {result[i]:.5f} (тыс.баррелей)')




Регион 0: Средний запас сырья на скважину 92.42384/предс/, 92.39349/реал/, RMSE 37.71690 (тыс.баррелей)
Регион 1: Средний запас сырья на скважину 68.98312/предс/, 68.98037/реал/, RMSE 0.89149 (тыс.баррелей)
Регион 2: Средний запас сырья на скважину 95.11622/предс/, 94.54782/реал/, RMSE 39.97554 (тыс.баррелей)


**Промежуточный вывод:**<br>
Метрика **RMSE** (средневкадратичное отклонение) исслюстрирует насколько хорошо модель соответствует набору данных. Чем больше RMSE, тем больше разница между прогнозируемыми и наблюдаемыми значениями, а это означает, что модель хуже соответствует данным. И наоборот, чем меньше RMSE, тем лучше модель соответствует данным. Таким образом мы видим, что в нашем проекте наилучшее предсказание получилось по данным региона 1, значение RMSE минимальное.  По регионам 0 и 2 среднеквадративное отклонение значительно выше, хотя средние значения предсказанных запасов сырья в целом близки к реальным данным.
    

<div class="alert alert-block alert-success">
<b>Успех:</b> Шаг проделан лаконично, все метрики посчитаны корректно
</div> 


# Шаг 3. Подготовка к расчёту прибыли 
Все ключевые значения для расчётов сохраним в отдельных переменных.<br>
Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. <br>
Сравним полученный объём сырья со средним запасом в каждом регионе.

In [15]:
# Ключевые значения для расчетов:

TOTAL_DRILLS_NUMBER = 500 # При разведке региона исследуют 500 точек
BEST_DRILLS_NUMBER = 200 # из которых с помощью машинного обучения выбирают 200 лучших для разработки
BUDGET_FOR_REGION = 1e10  # Бюджет на разработку скважин в регионе — 10 млрд рублей.
ONE_UNIT_PRICE = 450_000 #Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.


<div class="alert alert-block alert-info">
    
<b>Совет:</b> Константы принято записывать в ВЕРХНЕМ РЕГИСТРЕ =) Читем здесь https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#section-29
    
А большие значение можно записывать так: `450_000`.
</div> 


<div class="alert alert-block alert-warning">
<b>Изменения:</b> Константы записаны в верхнем регистре
</div>


<div class="alert alert-block alert-success">
<b>Успех[2]:</b> Так намного лучше)

In [16]:
# Для безубыточной разработки скважины доход должен превышать расходы, т.е. (объем)х(доход с единицы)>(бюждет на разработку)
# Объем безубыточности рассчитаем в тысячах баррелей, это бюджет поделенный на стоимость единицы

# При этом вспомним, что по условию бюджет разработки дан на регион в целом, то есть его нужно еще поделить на кол-во скважин

profitable_value = BUDGET_FOR_REGION / ONE_UNIT_PRICE / BEST_DRILLS_NUMBER

print(f'Достаточный объем сырья для окупаемости разработки скважин в регионе {profitable_value:.3f} тыс.баррелей с одной скважины')

Достаточный объем сырья для окупаемости разработки скважин в регионе 111.111 тыс.баррелей с одной скважины


In [17]:
# Cопоставим этот объем с данными по скважинам. 


for i in range(len(regions)):
    print(f'В регионе {i} скважина в среднем имеет дебит {zapas_pred[i]:.3f} тыс. баррелей')
print(f'При расчетном достаточном объеме для окупаемости разработки  {profitable_value:.3f} тыс.баррелей с одной скважины')

В регионе 0 скважина в среднем имеет дебит 92.424 тыс. баррелей
В регионе 1 скважина в среднем имеет дебит 68.983 тыс. баррелей
В регионе 2 скважина в среднем имеет дебит 95.116 тыс. баррелей
При расчетном достаточном объеме для окупаемости разработки  111.111 тыс.баррелей с одной скважины


**Промежуточный вывод:** <br><br>
По предварительной оценке, если **случайным** образом пробурить 200 скважин, ни в одном регионе разработка не окупится, т.к. в среднем одна скважина не обеспечивает необходимый объем сырья. <br>
Однако, если рассмотреть **лучшие по дебиту скважины**, ситуация может измениться.

<div class="alert alert-block alert-success">
<b>Успех:</b> Значение для безубыточной разработки посчитано верно, с выводом согласен  
</div>

# Шаг 4. Написание функции для расчета валовой прибыли 
Выберем скважины с максимальными значениями предсказаний.<br>
Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.<br>
Рассчитаем валовую прибыль для полученного объёма сырья.<br>


In [18]:
# Объявляем функцию, которая на вход получит датасет с данными геологоразведки, 
# а также все необхожимые для расчета прогноза валовой прибыли данные

# region_predict  - список предсказаных объемов
# n - число лучших скважин, в которых будет осуществляться добыча
# expenses - затраты на разработку в регионе
# price - цена за единицу объема, указанного в region_predict
# real_value - реальные данные о дебите скважим




def oil_profit(region_predict, n, expenses, price, real_value):
    
      
    # Сортируем предсказания, сохраняя индексы
    value_sort = pd.Series(region_predict).sort_values(ascending=False)

    # Определяем реальные показатели прибыли из реальных данных по индексам, суммируем их
    region_n_best_value = real_value.iloc[value_sort.index][:n].sum()
    
    # выручка это произвежение общего объема на цену за единицу
    income = region_n_best_value * price
    
    # валовая прибыль это доходы минус расходы
    profit = income - expenses
    
    
    return profit
    

<div class="alert alert-block alert-success">
<b>Успех:</b> Ф-ция для подсчета прибыли определена верно
</div>


In [19]:
# Вызовем нашу функцию для расчета прибыли по трем регионам



for i in range(len(regions)):

    profit = oil_profit(predictions_valid[i], BEST_DRILLS_NUMBER, BUDGET_FOR_REGION, ONE_UNIT_PRICE, target_valid[i])
    print(f'Регион {i} - прогноз прибыли с 200 лучших скважин (по предсказанным объемам): {profit/1e9:.4} млрд. руб.')
        
        

Регион 0 - прогноз прибыли с 200 лучших скважин (по предсказанным объемам): 3.136 млрд. руб.
Регион 1 - прогноз прибыли с 200 лучших скважин (по предсказанным объемам): 2.415 млрд. руб.
Регион 2 - прогноз прибыли с 200 лучших скважин (по предсказанным объемам): 2.466 млрд. руб.


**Промежуточный вывод:**<br>
Если располагать данными по 2500 разведочным скважинам (выборка valid), то из них вполне можно найти 200 лучших в каждом регионе, которые позволят получить валовую прибыль 2,5-3,8 млрд.рублей. Однако, если верить условиям задачи, в каждом регионе может быть разведано только 500 скважин, из которых и нужно будет определить лучшие 200. При таком "раскладе" шанс не получить прибыль сильно выше, т.к. мы видели выше, что в среднем случайная скважина не обеспечивает необходимый для прибыли дебит, а значит результат сильно чувствитетелен к тому, попадут ли в выборку из 500 скважин те лучшие, которые имеют хороший показатель прогнозных объемов и 200 которых смогут обеспечить прибыль.

<br><br>

# Шаг 5. Расчёт прибыли и рисков
Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.<br>
Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков.<br>
По итогам предложим регион для разработки скважин.


In [20]:
# Проводим численный эксперимент:
# 1. Из исходного датасета случайным образом формируем 1000 выборок по 500 строк, без возвращения (replace=False)
#    используя модель Линейной регрессии (обученную на всей исходной выборке) прогнозируем объем по 500 скважинам
# 2. На каждом шаге по 200 лучшим из 500-та выбранных скважин рассчитываем валовую прибыль
# 3. Определяем среднюю прибыль c 200 лучших скважин и 95% доверительный интервал, риск убытков.

# указываем random_state для воспроизводимости
state = np.random.RandomState(42)

# отдельно введем переменную с количеством итераций Bootstrap
boot = 1000

for j in range(len(regions)):

    values = []

    for i in range(boot):
        # извлекаем из выборки 500 (хранится в константе TOTAL_DRILLS_NUMBER) случайных строк
        subsample_features = features_valid[j].sample(TOTAL_DRILLS_NUMBER, replace=True, random_state=state)
        
        # находим реальные значения дебита скважин для расчета прибыли
        subsample_target = target_valid[j].iloc[subsample_features.index]
        
        # по выборке 500 скважин строим предсказание, используя ранее обученную модель и вызываем функцию расчета прибыли
        values.append(oil_profit(model[j].predict(subsample_features), BEST_DRILLS_NUMBER, BUDGET_FOR_REGION, ONE_UNIT_PRICE, subsample_target))

    # расчитываем границы 95% доверительного интервала (квантиль 2,5% снизу и 97,5% сверху)
    values = pd.Series(values)
    lower = values.quantile(q=0.025)
    upper = values.quantile(q=0.975)    

    # Расчет риска убытков. Вероятность получения убытков - это вероятность отрицательной прибыли.
    # Рассчитаем количество отрицательных значений прибыли и поделим на количество экспериментов
    neg_count = sum(1 for x in values if x < 0)
    risk_prob = neg_count / boot *100
    
    
    # выводим границы диапазона и среднее значение прибыли по регионам
    print('Регион', j)
    print(f'Ожидаемая прибыль {values.mean()/1e9:.3f} млрд. руб., с вероятностью 95% в диапазоне от {lower/1e9:.3f} до {upper/1e9:.3f} млрд.руб.')
    print(f'Вероятность (риск) получения убытка в регионе {risk_prob:.1f}%')
    print()


Регион 0
Ожидаемая прибыль 0.426 млрд. руб., с вероятностью 95% в диапазоне от -0.098 до 0.954 млрд.руб.
Вероятность (риск) получения убытка в регионе 5.7%

Регион 1
Ожидаемая прибыль 0.463 млрд. руб., с вероятностью 95% в диапазоне от 0.045 до 0.873 млрд.руб.
Вероятность (риск) получения убытка в регионе 1.2%

Регион 2
Ожидаемая прибыль 0.328 млрд. руб., с вероятностью 95% в диапазоне от -0.157 до 0.846 млрд.руб.
Вероятность (риск) получения убытка в регионе 10.5%




<div class="alert alert-block alert-danger">
<b>Ошибка:</b>

Особенностью техники бутстреп является семплирование с возвратом (`replace = True`, которое нужно явно указать), то есть наблюдение в выборку может попасть несколько раз. Это необходимо для того, чтобы добиться независимости при каждом выборе, иначе вероятность достать элемент из выборки будет с каждым разом увеличиваться. Причем чем больше размер подвыборки, тем выраженнее будет зависимость. Для подвыборки размера 1, например, в крайнем случае, мы возрат можем не делать, так уж и быть, а вот для подвыборки размером с семплируемую, без возврата ну никак не получится получить разнообразные варианты, а подвыборку большего размера вообще — не получить.
    
Все это может показаться немного странным для понимания: как будто одна скважина может буриться несколько раз. Но давай вспомним, что выборка - это лишь некоторое представление о генеральной совокупности. Вполне возможно, что где-то в регионе существуют и другие потенциальные точки добычи, о которых мы даже не подозреваем, но они являются частью генеральной сововкупности. В таком случае повторяющиеся наблюдения можно рассматривать как разные точки добычи из всей генеральной совокупности, просто они имеют близкие характеристики.
</div>


<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения
    
Применено семплирование с возвратом. Первоначально неверное понял смысл параметра replace: думал, что запрещая возврат (=False) мы запрещаем повторы наблюдений <b>в рамках одной случайно выборки</b>, то есть исключаем ситуацию, когда модель обучается на выборке, в которой несколько раз дублируется наблюдение. Спасибо за комментарий!

</div>


<div class="alert alert-block alert-success">
    
<b>Ответ[2]:</b> `то есть исключаем ситуацию, когда модель обучается на выборке, в которой несколько раз дублируется наблюдение` - как раз таки, в рамках одной выбоки нам такое исключать и не нужно, второй абзац объясняет это простыми словами)

**Промежуточный вывод**<br>
Анализ методом Bootstrap показал, что если случайным образом исследовать 500 точек в регионе и выбрать 200 наилучших предсказаний по прибыльности, то в 95% вероятностью прибыль по региону будет лежать в указанных выше диапазонах, и в том числе может достигать отрицательных значений (то есть будет убыток). Веротяность получения убытков также рассчитан.<br>
На основе проведенных расчетов предлагается остановить выбор на "Регионе 1", ожидаемая прибыль в котором составит 0,473 млрд.руб., при риске получения убытков 0,6%. Этот регион демонстрирует максимальную ожидаемую прибыль при минимальных рисках убытков.

# Выводы по проекту

В проекте был проведен анализ данных геологоразведки в трех регионах под условными номерами 0, 1 и 2.<br>

1. Было предоставлено три идентичных набора данных, для удобства и оптимизации кода все данные были объеденены в один объект - словарь. Изменяя лишь переменную с номерами файлов можно масштабировать проект до любого адекватного количества данных о скважинах. 

Данные были загружены, проверены на полноту и сотоветствие типов, несущественные данные (id) удалены. Потребовалось провести масштабирование, которое было проведено после раздедения данных на выборки, во избежание эффекта утечки данных. <br>
### UPD
Проведена оценка корреляции признаков. По региону "2" всё отлично, корреляции минимальные. В регионе "0" есть заметная отрицательная корреляция признаков f0 и f1 (-0.44), а также признака f2 и целевого признака product (0.48). 
А вот с датасетов региона "1" ситуация сложнее. Высокая корреляция признака f2 и целевого признака product, значение практически единица. Это говорит о том, что имеется сильно выраженная прямопропорциональная зависимость между признаком f2 и целевым признаком. **Но по условиям проекта - все признаки значимы и удалять мы их не будем.**



2. Данные разделены на обучающую и валидационную выборки в соотношении 75:25.<br>
Обучение моделей проведено отдельно для каждого набора данных по регионам геологоразведки на обучаюзщих выборках, предсказания выполнены на валидационных выборках.<br>
Рассчитан средний запас предсказанного сырья и RMSE моделей:<br>

Регион 0: Средний запас сырья на скважину 92.42384/предс/, 92.39349/реал/, RMSE 37.71690 (тыс.баррелей)<br>
Регион 1: Средний запас сырья на скважину 68.98312/предс/, 68.98037/реал/, RMSE 0.89149 (тыс.баррелей)<br>
Регион 2: Средний запас сырья на скважину 95.11622/предс/, 94.54782/реал/, RMSE 39.97554 (тыс.баррелей)<br>

Метрика **RMSE** (средневкадратичное отклонение) исслюстрирует насколько хорошо модель соответствует набору данных. Чем больше RMSE, тем больше разница между прогнозируемыми и наблюдаемыми значениями, а это означает, что модель хуже соответствует данным. И наоборот, чем меньше RMSE, тем лучше модель соответствует данным. Таким образом мы видим, что в нашем проекте наилучшее предсказание получилось по данным региона 1, значение RMSE минимальное.  По регионам 0 и 2 среднеквадративное отклонение значительно выше, хотя средние значения предсказанных запасов сырья в целом близки к реальным данным.<br>

3. Проведена подготовка к расчёту прибыли. <br>
Все ключевые значения для расчётов сохранены в отдельных переменных.<br>
Рассчитан достаточный объём сырья для безубыточной разработки новой скважины, который составил 111.111 тыс.баррелей с одной скважины.<br>
Проведено сравнение полученного объёма сырья со средним запасом в каждом регионе: <br>
В регионе 0 скважина в среднем имеет дебит 92.424 тыс. баррелей<br>
В регионе 1 скважина в среднем имеет дебит 68.983 тыс. баррелей<br>
В регионе 2 скважина в среднем имеет дебит 95.116 тыс. баррелей<br>

Таким образом, по предварительной оценке, если бурить скважины **случайным** образом, ни в одном регионе разработка не окупится, т.к. в среднем одна скважина не обеспечивает необходимый объем сырья. <br>

Однако, если рассмотреть **лучшие по дебиту скважины**, ситуация может измениться.


4. Написана функции для расчета валовой прибыли. Для этого выбраны скважины с максимальными значениями предсказаний. Просуммированы целевые значения объёма сырья, соответствующие этим предсказаниям. Рассчитана валовая прибыль для полученного объёма сырья:<br>
Регион 0 - прогноз прибыли с 200 лучших скважин (по предсказанным объемам): 3.136 млрд. руб.<br>
Регион 1 - прогноз прибыли с 200 лучших скважин (по предсказанным объемам): 2.415 млрд. руб.<br>
Регион 2 - прогноз прибыли с 200 лучших скважин (по предсказанным объемам): 2.466 млрд. руб.<br>

Если располагать данными по 2500 разведочным скважинам (выборка valid), то из них вполне можно найти 200 лучших в каждом регионе, которые позволят получить валовую прибыль 2,5-3,8 млрд.рублей. Однако, если верить условиям задачи, в каждом регионе может быть разведано только 500 скважин, из которых и нужно  определить лучшие 200. При таком "раскладе" шанс не получить прибыль сильно выше, т.к. мы видели выше, что в среднем случайная скважина не обеспечивает необходимый для прибыли дебит, а значит результат сильно чувствитетелен к тому, попадут ли в выборку из 500 скважин те лучшие, которые имеют хороший показатель прогнозных объемов и 200 которых смогут обеспечить прибыль.

<br><br>

5. Проведен расчёт прибыли и рисков
Была применена технику Bootstrap с 1000 выборок, определено распределение прибыли.<br>
Найдена средняя прибыль, 95%-й доверительный интервал и риск убытков.<br>
По итогам предложен регион для разработки скважин.

Анализ методом Bootstrap показал, что если случайным образом исследовать 500 точек в регионе и выбрать 200 наилучших предсказаний по прибыльности, то в 95% вероятностью прибыль по региону будет лежать в указанных выше диапазонах, и в том числе может достигать отрицательных значений (то есть будет убыток). Веротяность получения убытков также рассчитан.<br>

### UPD
### На основе проведенных расчетов предлагается остановить выбор на "Регионе 1", ожидаемая прибыль в котором составит 0,436 млрд.руб., при риске получения убытков 1,2%. Этот регион демонстрирует максимальную ожидаемую прибыль при минимальных рисках убытков.





<div class="alert alert-block alert-info">
<b>Совет: </b> Не забудь обновить вывод, если потребуется.
</div>



<div class="alert alert-block alert-warning">
<b>Изменения:</b> Выводы обновлены (помечено UPD)

</div>


<div class="alert alert-block alert-success">
<b>Успех[2]:</b> Отлично, проведено детальное исследование и был выбран правильный регион!

<div class="alert alert-block alert-info">
<b>Совет: </b> Если интересно изучить тему бутстрапа глубже - я оставлю  ниже несколько интересных и полезных ссылок по теме:
        
1. "Bootstrap Hypothesis Testing in Statistics with Example" 
        
https://www.youtube.com/watch?v=9STZ7MxkNVg
        
2. "How to Calculate Bootstrap Confidence Intervals For Machine Learning Results in Python" 
        
https://machinelearningmastery.com/calculate-bootstrap-confidence-intervals-machine-learning-results-python/
        
3. "A Gentle Introduction to the Bootstrap Method" 

https://machinelearningmastery.com/a-gentle-introduction-to-the-bootstrap-method/
        
4. "An Introduction to the Bootstrap Method" 
        
https://towardsdatascience.com/an-introduction-to-the-bootstrap-method-58bcb51b4d60#:~:text=The%20basic%20idea%20of%20bootstrap,population%20mean)%20on%20sample%20data.&amp;text=A%20sample%20from%20population%20with%20sample%20size%20n
        
5. "Bootstrapping and Resampling in Statistics with Example" 
        
        
https://www.youtube.com/watch?v=O_Fj4q8lgmc
</div>
